[Accueil](../../../index.ipynb) > [Sommaire de Terminale](../../index.ipynb)

# Recherche textuelle : l'algorithme de Boyer Moore

La recherche d'un **motif** (pattern en anglais) dans une chaîne de caractère a de nombreuses applications concrètes :

- **Moteurs de recherche** : Dans les moteurs de recherche simples ou les outils de recherche locaux, Boyer-Moore peut être utilisé pour trouver des occurrences de mots-clés dans de grands volumes de texte.
- **Éditeurs de texte** : Les éditeurs de texte (Notepad++, Sublime Text, ...) utilisent des algorithmes comme Boyer-Moore pour implémenter la fonctionnalité de recherche rapide de mots ou de phrases.
- **Bioinformatique** : Recherche de séquences d'ADN ou alignement de séquences;
- **Détection de virus** : Les logiciels antivirus utilisent des algorithmes de recherche de motifs pour identifier des signatures de virus ou de malware dans des fichiers ou des paquets réseau;
- **Correcteurs orthographiques**
- **Interception de communications** (Echelon)
- ...

Comment savoir si une chaîne de caractère possède tel **motif**?

Essayons tout d'abord avec le premier algorithme qui vient à l'esprit (algorithme naïf).

## Algorithme naïf

Le motif _roseraie_ est-il présent dans la chaîne de caractère suivante : _Cueillez dès aujourd’hui les roses de la vie_ ?

Une approche naïve est la suivante:

|C|u|e|i|l|l|e|z| |d|è|s| |a|u|j|o|u|r|d|’|h|u|i| |l|e|s| |r|o|s|e|s| |d|e| |l|a| |v|i|e|
|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|
|r|o|s|e|r|a|i|e| | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | |

C et r sont différents on décale alors le mot _roseraie_

|C|u|e|i|l|l|e|z| |d|è|s| |a|u|j|o|u|r|d|’|h|u|i| |l|e|s| |r|o|s|e|s| |d|e| |l|a| |v|i|e|
|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|
| |r|o|s|e|r|a|i|e| | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | |

u et r ne correspondent pas,

on décale jusqu'à obtenir la première correspondance
 
|C|u|e|i|l|l|e|z| |d|è|s| |a|u|j|o|u|r|d|’|h|u|i| |l|e|s| |r|o|s|e|s| |d|e| |l|a| |v|i|e|
|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|
| | | | | | | | | | | | | | | | | | |**r**|o|s|e|r|a|i|e| | | | | | | | | | | | | | | | | | |

ici il y a bien correspondance entre _r_ et _r_, il faut donc comparer la lettre suivante. _d_ et _o_ ne correspondent pas, on continue le décalage...

|C|u|e|i|l|l|e|z| |d|è|s| |a|u|j|o|u|r|d|’|h|u|i| |l|e|s| |r|o|s|e|s| |d|e| |l|a| |v|i|e|
|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|
| | | | | | | | | | | | | | | | | | | | | | | | | | | | | |**r**|**o**|**s**|**e**|r|a|i|e| | | | | | | |

ici c'est le _s_ et le _r_ qui ne correspondent pas.

### Implémentation de l'algorithme naïf

Voici le pseudo code de l'Algorithme 

```
recherche_naive (Text, Pattern)
    mettre dans n la longueur de Text
    mettre dans m la longueur de Pattern
    initialiser le résultat à une liste vide
    Pour j = 0 à n − m (inclu)
        i <- 0;
        Tant que (i < m et Text[j+i] = Pattern[i])
            i := i + 1
        Fin Tant que
        Si i = m
            ajouter j dans le résultat
        Fin de Si
    Fin Pour
    Retourner le résultat
```
**Ajoutez** des *assert* afin de tester votre fonction.

Par exemple:
- recherche_mot("ATACGAGGGGGGGGGGGGGGGGGATAC", "ATAC") doit retourner [0, 23]
- recherche_mot("AAAAAAAAAAAAAAAAAAAAAAAAAAA", "TGTC") retourne []

**Implémentez** la fonction *search* qui prend en paramètre :
- le texte *text*;
- le motif *pattern*.

et qui retourne
- la liste des index de *text* pour lesquels il y a correspondance;


### Coût de cet algorithme.

On appelle $M$ la longeur du motif et $T$ la longeur du texte.

Dans le pire des cas on doit décaler $(T-M)$ fois, puis faire jusqu'à $M$ comparaisons.
On obtient donc un coût $(T-M)xM$ que l'on approximera à **$TxM$** car souvent $T>>M$.

Ainsi :

- plus le texte est grand plus le coût augmente, ce qui est normal
- plus le motif est long, plus le coût augmente.


**A FAIRE** : Ajouter un compteur de nombre de comparaisons et retourner cette valeur dans les deux cas.

Nous allons étudier maintentant un algorithme beaucoup plus efficace.

## Algorithme de Boyer-Moore

Pourquoi faire un algorithme plus rapide?

A cause de la taille des données à traiter : par exemple en bioinformatique, la recherche de [transcriptome](https://fr.wikipedia.org/wiki/Transcriptome) (plusieurs Mo) dans le génome humain (plusieurs Go).

### Un peu d'histoire

Cet algorithme a été développé en 1977 par :

<figure style="float:left; margin:10px 50px 10px 0px">
    <img src="img/rsb.jpg"
         alt="Robert S. Boyer"
         title="Robert S. Boyer"
         style="border:1px solid black;height:200px"
         >
    <figcaption><a href="https://en.wikipedia.org/wiki/Robert_S._Boyer">Robert S. Boyer</a> (1946-)</figcaption>
</figure>

<figure style="float:left; margin:10px 50px 10px 0px">
    <img src="img/jsm.jpg"
         alt="J Strother Moore"
         title="J Strother Moore"
         style="border:1px solid black;height:200px"
         >
    <figcaption><a href="https://en.wikipedia.org/wiki/J_Strother_Moore">J Strother Moore</a></figcaption>
</figure>

<div style="clear:both" class="alert alert-info">
    Nous allons utiliser une version simplifiée de l'algorithme de Boyer-Moore. Il s'agit de l'algorithme de Horspool publié en 1980.
</div>

## Principe sur un exemple

Nous allons tout d'abord expliquer le principe de l'algorithme sur un exemple de séquence d'ADN.

L'ADN est composé d'une séquence de 4 bases azotées:

- Adénine (A)
- Cytosine (C)
- Guanine (G)
- Thymine (T)

Soit la séquence suivante : _CAATGTCTGCACCAAGAGGCCGGCAGGT_

Le motif _CGGCAG_ est-il présent dans la chaîne ?

On détermine d'abord la position minimale de chaque lettre (sauf la dernière) du motif en partant par la droite du motif.

Ce qui donne:

|C|G|A|
|-|-|-|
|2|3|1|

**Dans cet algorithme on regarde tout d'abord la dernière lettre du motif, puis on remonte vers la gauche.**

|i     |0 |1 |2 |3 |4 |5         |6 |7 |8 |9 |10|11|12|13|14|15|16|17|18|19|20|21|22|23|24|25|26|27|
|-     |- |- |- |- |- |-         |- |- |- |- |- |- |- |- |- |- |- |- |- |- |- |- |- |- |- |- |- |- |
|chaîne|C |A |A |T |G |~~**T**~~ |C |T |G |C |A |C |C |A |A |G |A |G |G |C |C |G |G |C |A |G |G |T |
|motif |C |G |G |C |A |~~**G**~~ |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |

- Y a t-il correspondance ? -> NON
  - La lettre _T_ est-elle présente dans notre motif -> NON
    - On saute donc de la longeur du motif (saut maximal)

|i     |0 |1 |2 |3 |4 |5 |6 |7 |8 |9 |10|11        |12|13|14|15|16|17|18|19|20|21|22|23|24|25|26|27|
|-     |- |- |- |- |- |- |- |- |- |- |- |-         |- |- |- |- |- |- |- |- |- |- |- |- |- |- |- |- |
|chaîne|C |A |A |T |G |T |C |T |G |C |A |~~**C**~~ |C |A |A |G |A |G |G |C |C |G |G |C |A |G |G |T |
|motif |  |  |  |  |  |  |C |G |G |C |A |~~**G**~~ |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |
  

- Y a t-il correspondance ? -> NON
  - La lettre _C_ est-elle présente dans notre motif -> OUI
    - On saute donc de 2 pour les faire correspondre

|i     |0 |1 |2 |3 |4 |5 |6 |7 |8 |9 |10|11                                  |12|13        |14|15|16|17|18|19|20|21|22|23|24|25|26|27|
|-     |- |- |- |- |- |- |- |- |- |- |- |-                                   |- |-         |- |- |- |- |- |- |- |- |- |- |- |- |- |- |
|chaîne|C |A |A |T |G |T |C |T |G |C |A |**<span style="color:red">C</span>**|C |~~**A**~~ |A |G |A |G |G |C |C |G |G |C |A |G |G |T |
|motif |  |  |  |  |  |  |  |  |C |G |G |**<span style="color:red">C</span>**|A |~~**G**~~ |  |  |  |  |  |  |  |  |  |  |  |  |  |  |

- Y a t-il correspondance ? -> NON
  - La lettre _A_ est-elle présente dans notre motif -> OUI
    - On saute donc de 1 pour les faire correspondre

|i     |0 |1 |2 |3 |4 |5 |6 |7 |8 |9 |10|11|12|13                                   |14        |15|16|17|18|19|20|21|22|23|24|25|26|27|
|-     |- |- |- |- |- |- |- |- |- |- |- |- |- |-                                    |-         |- |- |- |- |- |- |- |- |- |- |- |- |- |
|chaîne|C |A |A |T |G |T |C |T |G |C |A |C |C |**<span style="color:red">A</span>** |~~**A**~~ |G |A |G |G |C |C |G |G |C |A |G |G |T |
|motif |  |  |  |  |  |  |  |  |  |C |G |G |C |**<span style="color:red">A</span>** |~~**G**~~ |  |  |  |  |  |  |  |  |  |  |  |  |  |


- Y a t-il correspondance ? -> NON
  - La lettre _A_ est-elle présente dans notre motif -> OUI
    - On saute donc de 1 pour les faire correspondre

|i     |0 |1 |2 |3 |4 |5 |6 |7 |8 |9 |10|11|12|13    |14                                   |15    |16|17|18|19|20|21|22|23|24|25|26|27|
|-     |- |- |- |- |- |- |- |- |- |- |- |- |- |-     |-                                    |-     |- |- |- |- |- |- |- |- |- |- |- |- |
|chaîne|C |A |A |T |G |T |C |T |G |C |A |C |C |~~A~~ |**<span style="color:red">A</span>** |**G** |A |G |G |C |C |G |G |C |A |G |G |T |
|motif |  |  |  |  |  |  |  |  |  |  |C |G |G |~~C~~ |**<span style="color:red">A</span>** |**G** |  |  |  |  |  |  |  |  |  |  |  |  |

- Y a t-il correspondance ? -> OUI
  - On décale sur la gauche
  - Y a t-il correspondance ? -> OUI
    - On décale sur la gauche
    - Y a t-il correspondance ? -> NON
      - La lettre _A_ est-elle présente dans notre motif -> OUI
        - On saute de 1

<div class="alert alert-info">On voit ici les limites de l'algorithme simplifié : le décalage de 1 ne fait pas correspondre les lettres.</div>

|i     |0 |1 |2 |3 |4 |5 |6 |7 |8 |9 |10|11|12|13|14|15|16                                      |17|18|19|20|21|22|23|24|25|26|27|
|-     |- |- |- |- |- |- |- |- |- |- |- |- |- |- |- |- |-                                       |- |- |- |- |- |- |- |- |- |- |- |
|chaîne|C |A |A |T |G |T |C |T |G |C |A |C |C |A |A |G |**~~<span style="color:red">A</span>~~**|G |G |C |C |G |G |C |A |G |G |T |
|motif |  |  |  |  |  |  |  |  |  |  |  |C |G |G |C |A |**~~<span style="color:red">G</span>~~**|  |  |  |  |  |  |  |  |  |  |  |

- Y a t-il correspondance ? -> NON
  - La lettre _A_ est-elle présente dans notre motif -> OUI
    - On saute donc de 1 pour les faire correspondre

|i     |0 |1 |2 |3 |4 |5 |6 |7 |8 |9 |10|11|12|13|14|15    |16    |17                                   |18|19|20|21|22|23|24|25|26|27|
|-     |- |- |- |- |- |- |- |- |- |- |- |- |- |- |- |-     |-     |-                                    |- |- |- |- |- |- |- |- |- |- |
|chaîne|C |A |A |T |G |T |C |T |G |C |A |C |C |A |A |~~G~~ |**A** |**<span style="color:red">G</span>** |G |C |C |G |G |C |A |G |G |T |
|motif |  |  |  |  |  |  |  |  |  |  |  |  |C |G |G |~~C~~ |**A** |**<span style="color:red">G</span>** |  |  |  |  |  |  |  |  |  |  |


- Y a t-il correspondance ? -> OUI
  - On décale sur la gauche
  - Y a t-il correspondance ? -> OUI
    - On décale sur la gauche
    - Y a t-il correspondance ? -> NON
      - La lettre _G_ est-elle présente dans notre motif -> OUI
        - On saute de (3-2=**1**)  (G en position 3, mais on était décalé de 2)

|i     |0 |1 |2 |3 |4 |5 |6 |7 |8 |9 |10|11|12|13|14|15|16|17    |18                                 |19|20|21|22|23|24|25|26|27|
|-     |- |- |- |- |- |- |- |- |- |- |- |- |- |- |- |- |- |-     |-                                  |- |- |- |- |- |- |- |- |- |
|chaîne|C |A |A |T |G |T |C |T |G |C |A |C |C |A |A |G |A |~~G~~ |**<span style="color:red">G</span>** |C |C |G |G |C |A |G |G |T |
|motif |  |  |  |  |  |  |  |  |  |  |  |  |  |C |G |G |C |~~A~~ |**<span style="color:red">G</span>** |  |  |  |  |  |  |  |  |  |

- Y a t-il correspondance ? -> OUI
  - On décale sur la gauche
  - Y a t-il correspondance ? -> NON
    - La lettre _G_ est-elle présente dans notre motif -> OUI
      - On saute de (3-1=**2**)  (G position 3, mais on était décalé de 1)

|i     |0 |1 |2 |3 |4 |5 |6 |7 |8 |9 |10|11|12|13|14|15|16|17                                   |18|19|20        |21|22|23|24|25|26|27|
|-     |- |- |- |- |- |- |- |- |- |- |- |- |- |- |- |- |- |-                                    |- |- |-         |- |- |- |- |- |- |- |
|chaîne|C |A |A |T |G |T |C |T |G |C |A |C |C |A |A |G |A |**<span style="color:red">G</span>** |G |C |**~~C~~** |G |G |C |A |G |G |T |
|motif |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |C |G |**<span style="color:red">G</span>** |C |A |**~~G~~** |  |  |  |  |  |  |  |

etc... etc...

|C|A|A|T|G|T|C|T|G|C|A|C|C|A|A|G|A|C|G|C|C|G|G|C|A|G|G|T|
|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|
| | | | | | | | | | | | | | | | | | | | |**C**|**G**|**G**|**C**|**A**|**G**| | |

- Y a t-il correspondance ? -> OUI
  - On décale à gauche
  - Y a t-il correspondance ? -> OUI
    - On décale à gauche
    - ...
    
**Le motif est donc présent dans notre chaîne.**

<video controls src="Boyer-Moore.mp4" style="border:1px solid black">
    
Exemple de cette vidéo : texte = ATAACAGGAGTAAATAACGGCTGGAGTAAAT et motif = CGGCTC

## Principe synthétique

- Prétaitement du motif : réalisation de la table de sauts
- On bouche jusqu'à la fin de la chaine
  - On boucle sur le motif en partant de la fin tant qu'il y a correspondance
    - Si on arrive à la fin du motif, le motif existe
    - Sinon, on regarde dans la table des sauts si la lettre est présente
      - Si la lettre est présente on effectue le saut correspondant **- le décalage effectué** si celui est positif, sinon 1
      - Sinon, on effectue le saut maximal
 
Voir https://en.wikipedia.org/wiki/Boyer%E2%80%93Moore%E2%80%93Horspool_algorithm

### Prétraitement du motif

Le but est d'obtenir l'écart minimal entre une lettre du motif et la fin du motif (la dernière lettre est omise)
Pourquoi minimal ? car la même lettre peut apparaitre plusieurs fois.

Ainsi pour notre exemple:'CGGCAG' on doit obtenir {'C':2, 'G':3, 'A':1}

Pour "rappel" on doit obtenir {'r':5,'a':4,'p':2,'e:1}

**Implémentation en python**

In [4]:
#Commencez par écrire des assert

def shift(pattern):
    """
    Retourne le dictionnaire des sauts pour les différentes lettres du motif (on exclu la dernière lettre)"""
    pass



<div class="alert alert-info">
    L'implémentation de l'algorithme de Horspool est quasi identique à celle de l'algorithme naïf sauf que l'on décale selon la table des sauts si le caractère est présent, de la longueur du motif sinon et que l'on parcourt le motif de droite à gauche.
</div>

In [4]:
def cherche_motif(text, pattern):
    """
    Cherche un motif dans un texte
    Retourne Vrai si le motif est trouvé, faux sinon
    """
    t = len(text)
    p = len(pattern)
    # Si le motif est plus long que le texte ou que le texte est vide, on retourne False
    if p == 0 or p>t:# Si le motif est vide ou +grand que le texte
        return False 
    # On crée la table des sauts
    jumps = shift(pattern)
    
    # On initialise l'indice i à 0
    i = 0
    while i<t-p:
        ################################################################
        # /!\ /!\ /!\ implémenter le reste /!\ /!\ /!\
        pass
        ################################################################
    
    return False

### vérification
motif = 'CGGCAG'
texte = 'CAATGTCTGCACCAAGACGCCGGCAGGTGCAGACCTTCGTTATAGGCGATGATTTCGAACCTACTAG'

table = pretraitement('CGGCAG')
print(table)
print(cherche_motif(texte, motif))

{'C': 2, 'G': 3, 'A': 1}
False


## Amélioration de la fonction

Ajouter un paramètre *mode* à votre fonction qui vaut 0 par défaut:

- par défaut votre fonction retourne True ou False
- avec le mode=1, votre fonction retourne une liste contenant les index du texte où le début du mot a été trouvé (une liste vide si aucune occurrence)


## Coût de cet algorithme

Ici le coût est beaucoup plus complexe à définir mais dans notre exemple il est évident qu'il est bien moindre que l'algorithme naïf.

Pour s'en convaincre, ajoutez un compteur de comparaisons.

Qui plus est, plus le motif est long plus l'algorithme est efficace.

## Exemple concret

Le *meilleur des mondes* de l'auteur *H. G. Wells* est disponible ici: https://www.gutenberg.org/cache/epub/60656/pg60656.txt

- Ecrire un programme qui télécharge le contenu de cette url et l'écrit dans un fichier.
- A l'aide de votre fonction *cherche_motif*
  - Comptez le nombre d'occurrences de 
    - 'invasion'
    - 'compositeur'
    - ' nsi '
  - Trouver l'emplacement des occurrences citées préalablement.
- Comparer le temps de traitement pour ne pas trouver l'occurrence *nsi* avec votre programme et avec l'algorithme naïf.


## webographie

- [Eduscol](https://cache.media.eduscol.education.fr/file/NSI/63/5/RA20_NSI_G_T_boyer-moore_1298635.pdf)
- [Algorithme de Boyer-Moore : Wikipedia](https://fr.wikipedia.org/wiki/Algorithme_de_Boyer-Moore)
- [Algorithme de Boyer-Moore-Horspool : Wikipedia](https://fr.wikipedia.org/wiki/Algorithme_de_Boyer-Moore-Horspool)


[Accueil](../../../index.ipynb) > [Sommaire de Terminale](../../index.ipynb)